In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import sys
import time
from RBC_Utilities import calcDoubletFraction, getInstrinsicViscosity, getStress

## Instrinsic Viscosity vs Ca (Angles)

In [ ]:
start_time = time.time()

timesteps = 2000
phis = [4.7, 3.8]
angles = [90-30*i for i in range(6)]
Ca_list = [(i+1)*0.01 for i in range(20)]
Ca_list += [round(i*0.0025+0.05,5) for i in range(20)]
Ca_list += [round(i*0.0025+0.02,5) for i in range(12)]
Ca_list += [round(i*0.0025+0.1025,5) for i in range(2)]
Ca_list = list(set(Ca_list))
Ca_list.sort()

intrinsic_eta = np.zeros((len(phis), len(Ca_list), len(angles),3))
for phi_index, phi in enumerate(phis):
    for Ca_index, Ca in enumerate(Ca_list):
        for angle_index, angle in enumerate(angles):
            result = getInstrinsicViscosity(phi, Ca, 0, timesteps, angle)
            intrinsic_eta[phi_index, Ca_index, angle_index, 0] = np.mean(result)
            intrinsic_eta[phi_index, Ca_index, angle_index, 1] = np.mean(result[-int(timesteps/2):])
            intrinsic_eta[phi_index, Ca_index, angle_index, 2] = np.mean(result[-int(timesteps/4):])
            
#intrinsic_eta = intrinsic_eta/len(angles)
for phi_index, phi in enumerate(phis):
    plt.figure(figsize = (16,12))
    for angle_index, angle in enumerate(angles):
        plt.plot(Ca_list, intrinsic_eta[phi_index, :, angle_index, 0], label = ('angle = '+str(angle)+'\nwhole time series'))
        #plt.plot(Ca_list, intrinsic_eta[phi_index, :, angle_index, 1], label = ('angle = '+str(angle)+'\nsecond half'))
        #plt.plot(Ca_list, intrinsic_eta[phi_index, :, angle_index, 2], label = ('angle = '+str(angle)+'\nlast quarter'))
    plt.xlabel('Ca', fontsize = 20)
    plt.ylabel(r'$\left[ \eta \right]$', fontsize = 20)
    plt.legend(prop={'size': 20})
    plt.title(r'$\left[ \eta \right]$'+'vs Ca - Two-cell system, phi = {}'.format(phis[phi_index]), fontsize = 20)
    plt.savefig("./Pictures/IntrinsicViscosity_TwoCellSystem_angles_WholeSeries_phi_{}.png".format(phis[phi_index]), dpi = 300)

print('Total time elapsed = ', time.time()-start_time)

## Elastic stress yx vs Ca (Angles)

In [ ]:
start_time = time.time()
stress_category_id = 0

phis = [4.7, 3.8]
angles = [90-30*i for i in range(6)]
Ca_list = [(i+1)*0.01 for i in range(20)]
Ca_list += [round(i*0.0025+0.05,5) for i in range(20)]
Ca_list += [round(i*0.0025+0.02,5) for i in range(12)]
Ca_list += [round(i*0.0025+0.1025,5) for i in range(2)]
Ca_list = list(set(Ca_list))
Ca_list.sort()

results = np.zeros((2, len(angles), len(Ca_list)))

for phi_index, phi in enumerate(phis):
    for Ca_index, Ca in enumerate(Ca_list):
        print("phi = {}, Ca = {}".format(phi, Ca))
        for angle_index, angle in enumerate(angles):
            results[phi_index, angle_index, Ca_index] = np.mean(getStress(phi, Ca, stress_category_id, 1000, 2000, angle)[:,3])

for phi_index, phi in enumerate(phis):
    plt.figure(figsize = (16,12))
    plt.title(stress_category[stress_category_id]+'\nTwo-cell system, phi = {}'.format(phis[phi_index]), fontsize = 30)
    for angle_index, angle in enumerate(angles):
        plt.plot(Ca_list, results[phi_index, angle_index,:], label = 'angle = {}'.format(angle))
    plt.legend(prop={'size': 15})
    plt.savefig("./Pictures/StressTensor_"+stress_category[stress_category_id]+"_TwoCell_angles_SecondHalf_phi_{}.png".format(phis[phi_index]), dpi = 300)    
print('Total time elapsed = ', time.time()-start_time)

## Doublet fraction vs Ca (Angles)

In [ ]:
# Make mean doublet fraction (ensemble) versus Ca plot here

start_time = time.time()

ncycle = 2000
phis = [4.7, 3.8]
Dms = [1.0]
Ts = [1.0]
text = ['Whole Series', 'Second half']

Ca_list = [round(i*0.0025+0.05,5) for i in range(20)]
Ca_list += [(i+1)*0.01 for i in range(20)]
Ca_list += [round(i*0.0025+0.02,5) for i in range(12)]
Ca_list += [round(i*0.0025+0.1025,5) for i in range(2)]
Ca_list = list(set(Ca_list))
Ca_list.sort()

angles = [90-30*i for i in range(6)]
dfa = np.zeros((len(phis), len(Dms)*len(Ts), len(Ca_list), len(angles), 2))
for phi_index, phi in enumerate(phis):
    for Ca_index, Ca in enumerate(Ca_list):
        print("phi = {}, Ca = {}".format(phi, Ca))
        for angle_index, angle in enumerate(angles):
            result = calcDoubletFraction(phi, Ca, Ts, Dms, 0, angle)
            dfa[phi_index, :, Ca_index, angle_index, 0] = result[:, 0]
            dfa[phi_index, :, Ca_index, angle_index, 1] = result[:, 1]
#dfa = dfa/len(angles) # average over the ensemble

for phi_index, phi in enumerate(phis):
    for i in range(2):
        plt.figure(figsize = (16,12))
        for angle_index, angle in enumerate(angles):
            for label_index in range(len(Dms)*len(Ts)):
                plt.plot(Ca_list, dfa[phi_index, label_index, :, angle_index, 0] 
                         ,label = 'angle = {}'.format(angle))

        plt.xlabel("Ca", fontsize = 20)
        plt.ylabel("Doublet Fraction", fontsize = 20)
        plt.legend(prop={'size': 15})
        plt.title("Two-cell system, phi = {}, {}".format(phi, text[i]), fontsize = 30)
        plt.savefig("./Pictures/TwoCellSystem_Doublet_Fraction_averageDF_vs_Ca_ncycle_{}_angles_phi_{}_{}.png".
                    format(ncycle, phi, text[i]), dpi = 300)

print('Total time elapsed = ', time.time()-start_time)